## 실습#1 우리학교는 어디에?

### (1) 라이브러리 설정

In [22]:
import sys
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

### (2) 데이터 수집하기(셀레니엄 활용)

In [43]:
# 웹드라이버의 주소를 적고, 웹사이트 불러오기
# driver = webdriver.Chrome('C:\chromedriver_win32\chromedriver')driver = webdriver.Chrome(
driver = webdriver.Chrome()
driver.get("http://krsearch.co.kr/")

In [44]:
# 학교 리스트
q_list = ['대건고등학교','관악고등학교','원곡고등학교','상록고등학교', '양지고등학교', '안산고등학교', '초지고등학교', '불곡고등학교']

In [50]:
# 고등학교 검색
all_data = []
for q in q_list:
    search_element = driver.find_element_by_name('p')
    search_element.clear()
    search_element.send_keys(q)
# 검색 후 클릭
    search_button = driver.find_element_by_class_name("searchbtn2")
    search_button.click()
# 검색결과 수집
    pageSource = driver.page_source
    bsObj = BeautifulSoup(pageSource, 'html.parser')
    school=[]
    data = bsObj.find_all('td',{"class":"bbslist"})
    school.append(data[0].get_text().strip())
    school.append(data[1].get_text().strip())
    driver.get("http://krsearch.co.kr/")
    all_data.append(school)

In [46]:
# 데이터 확인
all_data

[['대건고등학교', '대구광역시 달서구 월곡로94길 46 (월성동 247번지)'],
 ['관악고등학교', '서울특별시 영등포구 양평동 2가 23'],
 ['원곡고등학교', '경기도 안산시 단원구 원곡동 965-1번지'],
 ['상록고등학교', '경기도 안산시 상록구 본오동 산127-1번지'],
 ['양지고등학교', '경기도 안산시 단원구 고잔동 713번지'],
 ['안산고등학교', '경기도 안산시 상록구 청룡4길 8'],
 ['초지고등학교', '경기도 안산시 단원구 광덕1로 13'],
 ['불곡고등학교', '경기도 성남시 분당구 금곡로 149번지']]

In [48]:
# DataFrame 형태로 바꾸기
fdata = pd.DataFrame(all_data, columns = ['name','address'])

In [49]:
fdata

,name,address
0,대건고등학교,대구광역시 달서구 월곡로94길 46 (월성동 247번지)
1,관악고등학교,서울특별시 영등포구 양평동 2가 23
2,원곡고등학교,경기도 안산시 단원구 원곡동 965-1번지
3,상록고등학교,경기도 안산시 상록구 본오동 산127-1번지
4,양지고등학교,경기도 안산시 단원구 고잔동 713번지
5,안산고등학교,경기도 안산시 상록구 청룡4길 8
6,초지고등학교,경기도 안산시 단원구 광덕1로 13
7,불곡고등학교,경기도 성남시 분당구 금곡로 149번지


### (3) 위/경도 정보 만들기¶

In [51]:
import requests
import json

def get_latitude_longitude(addr):
    '''
    DESCRIPTION: 주소를 입력하면 위도(latitude)/경도(longitude)를 반환하는 함수 
    ex) get_latitude_longitude('경기도 안산시 상록구 한양대학로 55')
    '''
    location = addr
    response = requests.get('http://maps.googleapis.com/maps/api/geocode/json?sensor=false&language=ko&address=' + location)    
    loc_dict = json.loads(response.text)
    latitude = loc_dict['results'][0]['geometry']['location']['lat'] 
    longitude = loc_dict['results'][0]['geometry']['location']['lng']
    return [latitude, longitude]

In [52]:
# 위/경도 정보로 저장해 새로운 데이터 형성 후 저장하기
latitude_list = []
longitude_list = []
for i in fdata['address'].values:
    lat, lon = get_latitude_longitude(i)
    latitude_list.append(lat)
    longitude_list.append(lon)

fdata['latitude'] = latitude_list
fdata['longitude'] = longitude_list

In [53]:
fdata

,name,address,latitude,longitude
0,대건고등학교,대구광역시 달서구 월곡로94길 46 (월성동 247번지),35.832884,128.527009
1,관악고등학교,서울특별시 영등포구 양평동 2가 23,37.522265,126.883074
2,원곡고등학교,경기도 안산시 단원구 원곡동 965-1번지,37.333300,126.798053
3,상록고등학교,경기도 안산시 상록구 본오동 산127-1번지,37.299510,126.874971
4,양지고등학교,경기도 안산시 단원구 고잔동 713번지,37.309535,126.822268
5,안산고등학교,경기도 안산시 상록구 청룡4길 8,37.360044,126.873672
6,초지고등학교,경기도 안산시 단원구 광덕1로 13,37.309868,126.810919
7,불곡고등학교,경기도 성남시 분당구 금곡로 149번지,37.348080,127.119199


In [55]:
import folium
# 오류수정 17.10.10 - Latitude, Longitude를 latitude, longitude로 변경. 
hanyang_univ_map = folium.Map(location=[37.298524, 126.834712], zoom_start=10)
for l in fdata[['name', 'latitude', 'longitude']].values:
    folium.CircleMarker(l[1:], popup=l[0], color='red', fill_color='red', radius=5).add_to(hanyang_univ_map)

In [34]:
hanyang_univ_map

## 실습#2 제일 가까운 학교는?

### (1) 기준 위치 설정하기

In [59]:
# 주소를 기반으로 위/경도 데이터 얻기
my_location = get_latitude_longitude('경기도 안산시 상록구 사3동 한양대학로55 한양대학교 경상대학')
my_location = [37.296755, 126.836194]
my_location_name = '한양대학교 제4공학관'

### (2) 위치 계산기 만들기

In [60]:
import math
def haversine(my_location, target_location):
    '''
    DESCRIPTION: 두 위도경도 사이의 대원거리(great circle distance)를 계산. 위도경도는 각도 단위로 계산.
    '''
    latitude1, longitude1 = my_location
    latitude2, longitude2 = target_location
    lat1, lon1, lat2, lon2 = map(math.radians, [latitude1, longitude1, latitude2, longitude2])
    
    d_lat = lat2 - lat1
    d_lon = lon2 - lon1
    
    a = math.sin(d_lat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(d_lon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    
    meter = 6367 * c * 1000
    return round(meter, 2)

### (3) 거리 정보 저장하기

In [61]:
# 계산된 거리 정보 리스트화 하기
distance_list = []
for r in fdata[['latitude', 'longitude']].values:
    distance_list.append(haversine(my_location, r))

In [62]:
distance_list

[221884.94, 25399.13, 5277.85, 3441.52, 1879.41, 7773.66, 2667.32, 25651.42]

In [63]:
#기존의 school_df 데이터프레임 데이터에 정보 추가하기
fdata['distance'] = distance_list

In [64]:
fdata

,name,address,latitude,longitude,distance
0,대건고등학교,대구광역시 달서구 월곡로94길 46 (월성동 247번지),35.832884,128.527009,221884.94
1,관악고등학교,서울특별시 영등포구 양평동 2가 23,37.522265,126.883074,25399.13
2,원곡고등학교,경기도 안산시 단원구 원곡동 965-1번지,37.333300,126.798053,5277.85
3,상록고등학교,경기도 안산시 상록구 본오동 산127-1번지,37.299510,126.874971,3441.52
4,양지고등학교,경기도 안산시 단원구 고잔동 713번지,37.309535,126.822268,1879.41
5,안산고등학교,경기도 안산시 상록구 청룡4길 8,37.360044,126.873672,7773.66
6,초지고등학교,경기도 안산시 단원구 광덕1로 13,37.309868,126.810919,2667.32
7,불곡고등학교,경기도 성남시 분당구 금곡로 149번지,37.348080,127.119199,25651.42


In [65]:
#거리 순으로 데이터 정렬하기
fdata = fdata.sort_values('distance')
fdata.reset_index(drop=True, inplace=True)
fdata

,name,address,latitude,longitude,distance
0,양지고등학교,경기도 안산시 단원구 고잔동 713번지,37.309535,126.822268,1879.41
1,초지고등학교,경기도 안산시 단원구 광덕1로 13,37.309868,126.810919,2667.32
2,상록고등학교,경기도 안산시 상록구 본오동 산127-1번지,37.299510,126.874971,3441.52
3,원곡고등학교,경기도 안산시 단원구 원곡동 965-1번지,37.333300,126.798053,5277.85
4,안산고등학교,경기도 안산시 상록구 청룡4길 8,37.360044,126.873672,7773.66
5,관악고등학교,서울특별시 영등포구 양평동 2가 23,37.522265,126.883074,25399.13
6,불곡고등학교,경기도 성남시 분당구 금곡로 149번지,37.348080,127.119199,25651.42
7,대건고등학교,대구광역시 달서구 월곡로94길 46 (월성동 247번지),35.832884,128.527009,221884.94


In [66]:
#가장 가까운 곳 찾기
nearest_latlng = fdata.loc[0][['latitude', 'longitude']].values

### (4) 지도로 표시하기

In [67]:
# 지도 
hanyang_univ_map2 = folium.Map(location=[37.300853, 126.842930], zoom_start=13)
# 가까운 학교표시
folium.CircleMarker(nearest_latlng, popup=fdata.loc[0]['name'], color='red', fill_color='red', radius=5).add_to(hanyang_univ_map2)
# 내 위치 표시
folium.CircleMarker(my_location, popup=my_location_name,color='blue', fill_color='blue', radius=5).add_to(hanyang_univ_map2)
hanyang_univ_map2